<a href="https://colab.research.google.com/github/mariajmolina/UMDAOSC650/blob/main/firstgnn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Our first graph neural network!

A graph neural network (GNN) is useful any time your data is naturally a graph, i.e., things (nodes) connected by relationships (edges), and you want to learn from both the attributes of the things and the structure of their connections.

Check out for inspiration: https://arxiv.org/html/2410.12938v1

GNNs are most useful when:

1. Data live on irregular observation networks

*   Nodes: weather stations, buoys, radars, cities;
*   Edges: geographic proximity, or learned similarity (correlated temperature, wind, elevation, etc.). GNNs naturally handle non-uniform station layouts, unlike CNNs that assume a regular grid. Reviews and recent work emphasize this as a key motivation for using GNNs in short- to medium-range forecasting.

2. You need spatio-temporal dependence across sites. ST-GNNs can learn how signals propagate between locations over time (e.g., fronts, advection, local micro-climates).
Examples:

*   Weather forecasting from station networks
*   Frost and minimum-temperature prediction over an IoT network and nearby stations to predict minimum temperature and frost incidence.

3. Air-quality and environmental prediction. Air-quality stations form a natural graph; GNNs model pollutant transport and meteorological influence between sites.

4. Post-processing and bias correction of NWP / ensembles. GNNs can take coarse-resolution ensemble forecasts plus station observations, treat stations (or grid cells) as nodes, and learn spatially coherent correction fields, improving local forecast skill and spatial coherence.

5. High-resolution regional nowcasting and interpolation. For complex terrain or urban areas, GNNs can connect irregular observation sites and interpolate or nowcast fields (temperature, precipitation, wind) at high spatial and temporal resolution, providing an alternative to dense grids.

In short, technically:

*   Use a GNN when your atmospheric or climate data are naturally represented as nodes connected by physical or statistical relationships (stations, cities, grid cells, ensemble members).
*   Use a spatio-temporal GNN when you need to learn how signals evolve both across space (advection, teleconnections, micro-climates) and time, especially on irregular networks where CNNs on regular grids are a poor fit.

In [1]:
!pip install torch_geometric

from google.colab import drive
import os
import pandas as pd
import torch
import torch.nn as nn
from torch_geometric.nn import GCNConv

# if you have gpu, you can call it
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [2]:
# mount google drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
# If you put data in a folder called "datasets":
# double check the file names
os.listdir('/content/drive/My Drive/datasets')

['all_EasyWeatherPro-3C1929(202510271423-202511261423).xlsx',
 'chirps-v2.0.ethio.1981-2024.days_p05.nc',
 'Radar Data']

In [4]:
path = '/content/drive/My Drive/datasets/all_EasyWeatherPro-3C1929(202510271423-202511261423).xlsx'

In [5]:
df = pd.read_excel(path, header=[0, 1])  # load keeping two top rows as header

In [6]:
df # quick check

Time        Outdoor                                         \
    Unnamed: 0_level_1 Temperature(℉) Temperature Low(℉) Temperature High(℉)   
0     2025-10-27 16:00           90.1               84.0                98.6   
1     2025-10-27 20:00           82.6               81.5                84.0   
2     2025-10-28 00:00           80.5               79.7                81.3   
3     2025-10-28 04:00           79.5               79.2                80.4   
4     2025-10-28 08:00           87.9               80.4                95.7   
..                 ...            ...                ...                 ...   
174   2025-11-25 16:00           87.1               84.4                88.7   
175   2025-11-25 20:00           81.8               80.2                84.4   
176   2025-11-26 00:00           78.9               77.0                80.8   
177   2025-11-26 04:00           76.6               75.7                77.5   
178   2025-11-26 08:00           78.4               75.4                80.6   

                                                                             \
    Feels Like(℉) Dew Point(℉) Humidity(%) Humidity Low(%) Humidity High(%)   
0            96.9         71.6          57              33               77   
1            91.6         77.5          85              77               90   
2            86.5         77.8          91              89               93   
3            79.9         77.8          94              92               95   
4            97.2         75.1          68              46               92   
..            ...          ...         ...             ...              ...   
174          87.1         33.4          15              13               17   
175          81.8         31.2          16              15               17   
176          78.9         31.0          17              16               20   
177          76.6         28.6          17              16               18   
178          78.4         29.6          17              15               19   

            Indoor  ...            Wind                                   \
    Temperature(℉)  ... Wind Speed(mph) Wind Gust(mph) Wind Direction(º)   
0             92.0  ...             6.3           19.5               296   
1             89.9  ...             4.4           13.7               300   
2             88.8  ...             1.0            5.8               269   
3             87.9  ...             2.7            8.1               272   
4             86.4  ...             3.6           10.3               335   
..             ...  ...             ...            ...               ...   
174           84.4  ...             4.7           19.5                47   
175           84.8  ...             5.6           12.5                20   
176           84.6  ...             4.6           13.7                36   
177           83.9  ...             5.3           13.7                24   
178           82.5  ...             3.9           11.4                67   

                                              Pressure                     \
    10-minute Average Wind Direction(º) Relative(inHg) Relative Low(inHg)   
0                                     -          29.91              29.90   
1                                     -          29.96              29.93   
2                                     -          29.93              29.89   
3                                     -          29.91              29.88   
4                                     -          29.95              29.91   
..                                  ...            ...                ...   
174                                   -          29.84              29.79   
175                                   -          29.92              29.88   
176                                   -          29.89              29.87   
177                                   -          29.89              29.86   
178                                 

In [7]:
# how to select multi header columns
cols = [
    ('Time', 'Unnamed: 0_level_1'),
    ('Outdoor', 'Temperature(℉)'),
    ('Outdoor', 'Humidity(%)'),
    ('Indoor', 'Temperature(℉)'),
    ('Indoor', 'Humidity(%)'),
    ('Outdoor', 'Dew Point(℉)')
]
sub = df[cols]   # or df.loc[:, cols]

In [8]:
sub

Time        Outdoor                     Indoor              \
    Unnamed: 0_level_1 Temperature(℉) Humidity(%) Temperature(℉) Humidity(%)   
0     2025-10-27 16:00           90.1          57           92.0          54   
1     2025-10-27 20:00           82.6          85           89.9          67   
2     2025-10-28 00:00           80.5          91           88.8          70   
3     2025-10-28 04:00           79.5          94           87.9          70   
4     2025-10-28 08:00           87.9          68           86.4          69   
..                 ...            ...         ...            ...         ...   
174   2025-11-25 16:00           87.1          15           84.4          27   
175   2025-11-25 20:00           81.8          16           84.8          26   
176   2025-11-26 00:00           78.9          17           84.6          28   
177   2025-11-26 04:00           76.6          17           83.9          29   
178   2025-11-26 08:00           78.4          17           82.5          22   

         Outdoor  
    Dew Point(℉)  
0           71.6  
1           77.5  
2           77.8  
3           77.8  
4           75.1  
..           ...  
174         33.4  
175         31.2  
176         31.0  
177         28.6  
178         29.6  

[179 rows x 6 columns]

In [9]:
# here we define 2 nodes: 0 = outdoor, 1 = indoor
edge_index = torch.tensor([
    [0, 1],  # from
    [1, 0],  # to
], dtype=torch.long).to(device)

In [10]:
class SimpleSTGNN(nn.Module):
    """
    Simple spatio-temporal GNN for a 2-node graph (outdoor + indoor).

    Architecture
    -----------
    - Spatial layer:  one GCNConv applied independently at each time step.
      This performs message passing over the 2-node graph using edge_index.
    - Temporal layer: one-layer GRU applied over time to the OUTDOOR node
      embeddings only (node index 0).
    - Readout layer:  linear layer mapping the last GRU hidden state to a
      single scalar (e.g. next-step outdoor dew point).

    Expected inputs
    ---------------
    x : torch.Tensor, shape [T, N, F]
        Time series window of node features.
        - T = number of time steps in the input window
        - N = number of nodes (here N=2: 0=outdoor, 1=indoor)
        - F = number of features per node (e.g. [temperature, humidity])

    edge_index : torch.LongTensor, shape [2, E]
        Graph connectivity in COO format as used by PyTorch Geometric.
        For this toy example with 2 nodes we typically use:
            edge_index = tensor([[0, 1],
                                 [1, 0]])
        and rely on GCNConv(add_self_loops=True) to add self-loops.

    Output
    ------
    y_hat : torch.Tensor, scalar
        Predicted target in normalized units
        (you de-normalize outside this module).
    """

    def __init__(self, in_channels: int = 2,
                 gnn_hidden: int = 8,
                 rnn_hidden: int = 8) -> None:
        """
        Parameters
        ----------
        in_channels : int
            Number of input features per node F
            (e.g. 2 for [temperature, humidity]).

        gnn_hidden : int
            Dimensionality of the node embeddings produced by the GCN layer.

        rnn_hidden : int
            Dimensionality of the GRU hidden state (temporal representation).
        """
        super().__init__()

        # Spatial layer: graph convolution on a single snapshot of the graph.
        # add_self_loops=True means each node also receives its own features.
        self.gcn = GCNConv(in_channels, gnn_hidden, add_self_loops=True)

        # Temporal layer: GRU processes the sequence of outdoor embeddings
        # over time. batch_first=True expects input of shape [B, T, gnn_hidden].
        self.rnn = nn.GRU(gnn_hidden, rnn_hidden, batch_first=True)

        # Readout: map last hidden state to a single scalar prediction.
        self.readout = nn.Linear(rnn_hidden, 1)

    def forward(self, x: torch.Tensor,
                edge_index: torch.LongTensor) -> torch.Tensor:
        """
        Run the model on a single time window (no batch dimension).

        Parameters
        ----------
        x : torch.Tensor, shape [T, N, F]
            Input node features over T time steps.

        edge_index : torch.LongTensor, shape [2, E]
            Graph edges shared across all time steps.

        Returns
        -------
        torch.Tensor
            Scalar prediction for this window (e.g. next-step dew point).
        """
        T, N, F = x.shape
        assert N == 2, "This toy example assumes exactly 2 nodes (outdoor + indoor)."

        x = x.to(device)

        gnn_outputs = []  # will store outdoor embeddings at each time step

        # ---- Spatial part: apply GCN at each time step independently ----
        for t in range(T):
            # Features at time t for both nodes: shape [N, F]
            x_t = x[t]

            # Message passing on the 2-node graph: shape [N, gnn_hidden]
            h_t = self.gcn(x_t, edge_index)

            # Non-linearity
            h_t = torch.relu(h_t)

            # Keep only the outdoor node embedding (node index 0)
            gnn_outputs.append(h_t[0])  # shape [gnn_hidden]

        # Stack outdoor embeddings into a sequence: [T, gnn_hidden]
        # Then add a batch dimension B=1 → [1, T, gnn_hidden]
        seq = torch.stack(gnn_outputs, dim=0).unsqueeze(0)

        # ---- Temporal part: GRU over the sequence of outdoor embeddings ----
        # rnn_out: [1, T, rnn_hidden]
        rnn_out, _ = self.rnn(seq)

        # Take the hidden state at the final time step: [1, rnn_hidden]
        last_hidden = rnn_out[:, -1, :]

        # ---- Readout: map final temporal representation to scalar target ----
        # y_hat: [1, 1]
        y_hat = self.readout(last_hidden)

        # Return scalar (drop batch and feature dims)
        return y_hat.squeeze()

In [11]:
def load_and_prepare(sub: pd.DataFrame,
                     input_len: int = 24,
                     train_frac: float = 0.8):
    """
    Prepare a spatio-temporal dataset from a MultiIndex weather DataFrame.

    Parameters
    ----------
    sub : pd.DataFrame
        DataFrame with a 2-level column MultiIndex, e.g.:
            ('Time', 'Unnamed: 0_level_1')
            ('Outdoor', 'Temperature(℉)')
            ('Outdoor', 'Humidity(%)')
            ('Indoor', 'Temperature(℉)')
            ('Indoor', 'Humidity(%)')
            ('Outdoor', 'Dew Point(℉)')
        The function assumes these exact column tuples exist.

    input_len : int, default=24
        Length of the look-back window (number of time steps) used as input
        for each training sample.

    train_frac : float, default=0.8
        Fraction of samples used for training; the remainder is used as a
        chronological test set.

    Returns
    -------
    train_data : list[tuple[torch.Tensor, torch.Tensor]]
        List of (x_window, y_target) pairs for training.
        - x_window: torch.Tensor of shape [T, 2, 2]
            T = input_len
            Node dimension N=2 corresponds to:
                0: Outdoor  (features: temp, humidity)
                1: Indoor   (features: temp, humidity)
        - y_target: scalar torch.Tensor
            Normalized next-step outdoor dew point.

    test_data : list[tuple[torch.Tensor, torch.Tensor]]
        Same format as train_data, but for the held-out chronological tail.

    norm_params : dict
        Normalization statistics used for z-scoring:
        {
            "feat_mean": np.ndarray of shape [1, 4],
            "feat_std":  np.ndarray of shape [1, 4],
            "targ_mean": float,
            "targ_std":  float,
        }
        These are needed later to de-normalize predictions.
    """

    # Column tuples in the MultiIndex header
    cols = [
        ('Time', 'Unnamed: 0_level_1'),     # time stamp
        ('Outdoor', 'Temperature(℉)'),      # outdoor temperature
        ('Outdoor', 'Humidity(%)'),         # outdoor humidity
        ('Indoor', 'Temperature(℉)'),       # indoor temperature
        ('Indoor', 'Humidity(%)'),          # indoor humidity
        ('Outdoor', 'Dew Point(℉)')         # outdoor dew point (target)
    ]

    # ---------------------------------------------------------------------
    # 1. Time handling: parse and sort
    # ---------------------------------------------------------------------
    sub[cols[0]] = pd.to_datetime(sub[cols[0]])
    sub = sub.sort_values(cols[0]).reset_index(drop=True)

    # ---------------------------------------------------------------------
    # 2. Extract features and target as numpy arrays
    # ---------------------------------------------------------------------
    # 4 feature columns: outdoor (T, RH) + indoor (T, RH)
    feat_cols = [cols[1], cols[2], cols[3], cols[4]]
    # Target: outdoor dew point
    target_col = cols[5]

    feats = sub[feat_cols].values.astype('float32')    # [T_total, 4]
    target = sub[target_col].values.astype('float32')  # [T_total]

    # ---------------------------------------------------------------------
    # 3. Global normalization (z-score), simple but adequate for a demo
    # ---------------------------------------------------------------------
    feat_mean = feats.mean(axis=0, keepdims=True)
    feat_std = feats.std(axis=0, keepdims=True) + 1e-8
    targ_mean = target.mean()
    targ_std = target.std() + 1e-8

    feats_norm = (feats - feat_mean) / feat_std
    target_norm = (target - targ_mean) / targ_std

    # ---------------------------------------------------------------------
    # 4. Build sliding windows for supervised learning
    # ---------------------------------------------------------------------
    # Each sample uses 'input_len' past steps to predict the NEXT step
    # outdoor dew point (one-step-ahead forecasting).
    X_list, y_list = [], []

    T_total = len(sub)
    for i in range(input_len, T_total - 1):
        # Features for the look-back window: [input_len, 4]
        window_feats = feats_norm[i - input_len:i]
        window_feats = torch.tensor(window_feats, dtype=torch.float32)

        # Split feature columns into nodes:
        #   Node 0 (outdoor): cols 0–1  → [T, 2]
        #   Node 1 (indoor):  cols 2–3  → [T, 2]
        out_feats = window_feats[:, 0:2]   # [T, 2] outdoor (T, RH)
        in_feats  = window_feats[:, 2:4]   # [T, 2] indoor  (T, RH)

        # Stack nodes into [T, N, F] where N=2, F=2
        x_window = torch.stack([out_feats, in_feats], dim=1)  # [T, 2, 2]

        # Target is the normalized outdoor dew point at the *next* time step
        y_target = torch.tensor(target_norm[i + 1], dtype=torch.float32)

        X_list.append(x_window)
        y_list.append(y_target)

    # ---------------------------------------------------------------------
    # 5. Chronological train/test split
    # ---------------------------------------------------------------------
    num_samples = len(X_list)
    train_size = int(train_frac * num_samples)

    train_data = list(zip(X_list[:train_size], y_list[:train_size]))
    test_data  = list(zip(X_list[train_size:], y_list[train_size:]))

    # Store normalization parameters for later de-normalization
    norm_params = {
        "feat_mean": feat_mean,
        "feat_std": feat_std,
        "targ_mean": targ_mean,
        "targ_std": targ_std,
    }

    return train_data, test_data, norm_params

In [12]:
train_data, test_data, norm_params = load_and_prepare(df, input_len=170)

In SimpleSTGNN, in_channels is the number of features per node, not the total number of variables in the whole system.

In [13]:
model = SimpleSTGNN(in_channels=2, gnn_hidden=16, rnn_hidden=16).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)
loss_fn = nn.MSELoss()

In [14]:
# Number of passes over the full training set
num_epochs = 20

for epoch in range(num_epochs):
    # Put the model in training mode (enables dropout, etc. if present)
    model.train()

    total_loss = 0.0  # accumulates loss over all training samples in this epoch

    # ------------------------------------------------------------------
    # Iterate over all training samples
    # train_data is a list of (x_window, y_target) pairs:
    #   x_window: [T, 2, 2]  (time, node, feature)
    #   y_target: scalar     (normalized dew point)
    # ------------------------------------------------------------------
    for x_window, y_target in train_data:

        # Move data to the same device as the model (CPU or GPU)
        x_window = x_window.to(device)
        y_target = y_target.to(device)

        # Reset gradients from the previous iteration
        optimizer.zero_grad()

        # Forward pass: compute model prediction for this window
        #   y_pred is a scalar in normalized units
        y_pred = model(x_window, edge_index)

        # Compute loss between prediction and ground truth
        # Here loss_fn is typically nn.MSELoss()
        loss = loss_fn(y_pred, y_target)

        # Backward pass: compute gradients of loss w.r.t. model parameters
        loss.backward()

        # Gradient step: update model parameters using the optimizer
        optimizer.step()

        # Accumulate loss (as a Python float) for reporting
        total_loss += loss.item()

    # Average loss over all training samples in this epoch
    avg_loss = total_loss / len(train_data)

    # Note: this is in the normalized target space (z-scores), not physical units
    print(f"Epoch {epoch + 1:02d} | train loss (normalized units): {avg_loss:.4f}")

Epoch 01 | train loss (normalized units): 7.5762
Epoch 02 | train loss (normalized units): 6.8472
Epoch 03 | train loss (normalized units): 6.1473
Epoch 04 | train loss (normalized units): 5.4715
Epoch 05 | train loss (normalized units): 4.8157
Epoch 06 | train loss (normalized units): 4.1838
Epoch 07 | train loss (normalized units): 3.5850
Epoch 08 | train loss (normalized units): 3.0294
Epoch 09 | train loss (normalized units): 2.5238
Epoch 10 | train loss (normalized units): 2.0702
Epoch 11 | train loss (normalized units): 1.6671
Epoch 12 | train loss (normalized units): 1.3120
Epoch 13 | train loss (normalized units): 1.0038
Epoch 14 | train loss (normalized units): 0.7436
Epoch 15 | train loss (normalized units): 0.5328
Epoch 16 | train loss (normalized units): 0.3707
Epoch 17 | train loss (normalized units): 0.2529
Epoch 18 | train loss (normalized units): 0.1717
Epoch 19 | train loss (normalized units): 0.1181
Epoch 20 | train loss (normalized units): 0.0840


In [16]:
# Put the model in evaluation mode:
# - Disables dropout, batch-norm updates, etc.
# - Important to get deterministic behavior during evaluation.
model.eval()

# Retrieve normalization parameters for the target (dew point)
targ_mean = norm_params["targ_mean"]  # scalar mean of training dew point
targ_std  = norm_params["targ_std"]   # scalar std of training dew point

# Disable gradient tracking:
# - Reduces memory usage
# - Speeds up inference
# - Ensures we do not accidentally backpropagate through eval code.
with torch.no_grad():

    # Loop over a few examples from the test set
    # test_data is a list of (x_window, y_target) pairs, same format as train_data
    for x_window, y_target in test_data[:5]:  # just inspect first 5 samples

        # Move input window to the same device as the model
        x_window = x_window.to(device)

        # Forward pass: model returns prediction in *normalized* units
        # y_hat_norm is a scalar tensor; .cpu().item() converts to Python float
        y_hat_norm = model(x_window, edge_index).cpu().item()

        # y_target is already on CPU (we never moved it), and is also normalized
        y_true_norm = y_target.item()

        # --------------------------------------------------------------
        # De-normalize both prediction and ground truth back to
        # physical dew-point units (e.g., degrees F)
        # Recall: z = (y - mean) / std  ⇒  y = z * std + mean
        # --------------------------------------------------------------
        y_hat = y_hat_norm * targ_std + targ_mean   # predicted dew point
        y_true = y_true_norm * targ_std + targ_mean # true dew point

        # Print a simple side-by-side comparison
        print(f"pred dew point = {y_hat:.2f}, true = {y_true:.2f}")

pred dew point = 35.91, true = 28.60
pred dew point = 35.81, true = 29.60
